# Wind power station units

According to IEC 60909-0:2016, wind power station units are considered as impedance for the following types:
* Wind power station unit with asynchronous generator (6.8.2)
* Wind power station unit with doubly fed asynchronous generator (6.8.3)

Wind power station units with full size converter are considered as current source injections, and the contributions to the short-circuit current are specified by the datasheet of the manufacturer (6.9).

### Example grid

For all the further examples, we use the following grid model:

In [ ]:
import numpy as np
import pandapower.shortcircuit as sc
from pandapower.test.shortcircuit.test_sgen import simplest_test_grid as create_net

The grid model has one node, one ext_grid (with near 0 contribution, for the purpose of convergence) and a static generator:

$
X\\
|\\
\cdot\\
|\\
G\\
$

## Wind power station unit with asynchronous generator

The impedance $Z_G$ of the asynchronous generator is calculated as follows:


$Z_{WD} = \frac{1}{I_{LR} / I_{rG}} \cdot \frac{U_{rG}}{\sqrt{3} \cdot I_{rG}} = \frac{1}{I_{LR} / I_{rG}} \cdot \frac{U_{rG}^2}{S_{rG}}$

The complex value of $\underline{Z}_G$

$\underline{Z}_G = R_G + jX_G = (\frac{R_G}{X_G}+j) \cdot \frac{Z_G}{\sqrt{1 + (R_G/X_G)^2}}$

If $R_G/X_G$ is not provided by the manufacturer, then it can be assumed as 0.1

The total positive-sequence short-circuit impedance $\underline{Z}_{WA}$ of a wind power station unit with asynchronous generator for the calculation of the short-circuit current contribution on the high-voltage side of the unit transformer is calculated as follows:

$\underline{Z}_{WA} = t_r^2 \cdot \underline{Z}_G + \underline{Z}_{THV}$

### Example

Required parameters for sgen are:

* U_rG: rated voltage of the asynchronous generator (we take from the bus)
* I_rG: rated current of the asynchronous generator (calculated from S_rG and U_rG)
* S_rG: rated apparent power of the asynchronous generator (required input)
* R_G/X_G: R to X value of the asynchronous generator (required input)

Required new input parameter:
* I_LR/I_rG: the ratio of the symmetrical locked-rotor current to the rated current of the asynchronous generator (required new input, provided by the manufacturer)

In [ ]:
U_rG = 0.4  # kV
S_rG = 2.5 # MVA
I_rG = S_rG / (np.sqrt(3) * U_rG)
I_LR = 5 * I_rG # our assumption
I_LR_rG = I_LR / I_rG
RX_G = 0.1

In [ ]:
Z_G = 1/I_LR_rG * U_rG**2 / S_rG
Z_G

In [ ]:
Z_G_complex = (RX_G + 1j) * Z_G / (np.sqrt(1 + RX_G**2))
Z_G_complex

In [ ]:
t_r = 20/0.4
# transformer impedance is taken from the pandapower grid
# transformer from the example (see below)
Z_THV_complex = 2.3263781460713675+9.409936797295378j
Z_WA_complex = t_r ** 2 * Z_G_complex + Z_THV_complex
Z_WA_complex

In [ ]:
ikss_ka = 1.1 * 0.4 / (np.sqrt(3) * Z_G)
ikss_ka

In [ ]:
# skss
ikss_ka * 0.4 * np.sqrt(3)

In [ ]:
ikss_block_ka = 1.1 * 20 / (np.sqrt(3) * abs(Z_WA_complex))
ikss_block_ka

In [ ]:
# skss
ikss_block_ka * 20 * np.sqrt(3)

In [ ]:
kappa = 1.02 + 0.98 * np.exp(-3*0.1)
ikss_ka * kappa * np.sqrt(2)

In [ ]:
kappa_block = 1.02 + 0.98 * np.exp(-3 * Z_WA_complex.real / Z_WA_complex.imag)
ikss_block_ka * kappa_block * np.sqrt(2)

In [ ]:
%%capture
# now with pandapower:
net = create_net("async", step_up_trafo=True)
sc.calc_sc(net, ip=True)

In [ ]:
display(net.sgen)

In [ ]:
display(net.res_bus_sc)

## Wind power station units with doubly fed asynchronous generator

The total positive-sequence short-circuit impedance $Z_{WD}$ of a wind power station is defined as follows (eq. 28):

$Z_{WD} = \frac{\sqrt{2} \cdot \kappa_{WD} \cdot U_{rTHV}}{\sqrt{3} \cdot i_{WDmax}}$

where

$U_{rTHV}$ stands for the rated voltage of the unit transformer at the high-voltage side

$\kappa_{WD}$ is the factor for the caclulation of the peak short-circuit current, given by the manufacturer and referred to the high-voltage side (depends on the protection equipment, to be assumed 1.7 if not known)

$i_{WDmax}$ is the highest instantaneous short-circuit value in case of a three-phase short-circuit



The complex value of the short-circuit impedance $\underline{Z}_{WD}$ is calculated as follows (eq. 29):

$\underline{Z}_{WD} = R_{WD} + jX_{WD} = (\frac{R_{WD}}{X_{WD}}+j) \cdot \frac{Z_{WD}}{\sqrt{1+(R_{WD}/X_{WD})^2}}$

If $\frac{R_{WD}}{X_{WD}}$ is not provided by the manufacturer, then 0.1 can be assumed.

### Example

Required parameters for sgen are:

* U_rG: rated voltage of the asynchronous generator (we take from the bus)
* I_rG: rated current of the asynchronous generator (calculated from S_rG and U_rG)
* S_rG: rated apparent power of the asynchronous generator (required input)
* R_G/X_G: R to X value of the asynchronous generator (required input)

Required new input parameters:
* i_WDmax: the highest instantaneous short-circuit value in case of a three-phase short-circuit (required new input, provided by the manufacturer)
* k_WD: (kappa) the factor for the calculation of the peak short-circuit current, referred to the high-voltage side (required new input, provided by the manufacturer)

Calculation according to the example from IEC 60909-4:2021 (8.2, figure 16)

In [ ]:
S_rWd = 2.5 # MVA
U_rWD = U_rTWDHV = 20 # kV
i_WDmax = 0.388 # kA
# I_rWD is provided by the manufacturer
# i_kWDmax = 1.2 * I_rWD
k_WD = 1.7
RX_WD = 0.1

In [ ]:
Z_WD = np.sqrt(2) * k_WD * U_rTWDHV / (np.sqrt(3) * i_WDmax)
Z_WD # Ohm

In [ ]:
Z_WD_complex = (RX_WD + 1j) * Z_WD / np.sqrt(1 + RX_WD**2)
Z_WD_complex # Ohm

In [ ]:
ikss_ka = 1.1 * 20 / (np.sqrt(3) * Z_WD)
ikss_ka

In [ ]:
# skss
ikss_ka * 20 * np.sqrt(3)

In [ ]:
ikss_ka * k_WD * np.sqrt(2)

In [ ]:
%%capture
# now with pandapower:
net = create_net("async_doubly_fed")
sc.calc_sc(net, ip=True)

In [ ]:
display(net.sgen)

In [ ]:
display(net.res_bus_sc)

## Wind power station unit with full size converter

### Example

A typical range of short-circuit current contribution of a full size converter is 1.0 to 1.3 of the rated current $I_{rWF}$.
The wind power station unit with full size converter is considered as a current source, therefore the impedance is infinitely high:

$Z_{WF} = \infty$

Required input parameters:
* U_rG: rated voltage of the asynchronous generator (we take from the bus)
* I_rG: rated current of the asynchronous generator (calculated from S_rG and U_rG)
* S_rG: rated apparent power of the asynchronous generator (required input)
* R_G/X_G: R to X value of the asynchronous generator (required input)
* k: the factor of the short-circuit contribution of the full size converter in relation to its rated current (typically in the range between 1 and 1.3, depends on the requirements of the system operator)


In [ ]:
S_rWF = 2.5 # MVA
U_rWF = 20 # kV

In [ ]:
I_rWF = S_rWF / (np.sqrt(3) * U_rWF)
I_rWF # kA

In [ ]:
I_skWF = I_kWFmax = 1.3 * I_rWF
I_skWF # kA

In [ ]:
kappa = 1.02 + 0.98 * np.exp(-3*0.1)
I_skWF * np.sqrt(2) # because single power station unit with full size converter

In [ ]:
# skss
I_skWF * 20 * np.sqrt(3)

In [ ]:
%%capture
# now with pandapower:
net = create_net("current_source")
sc.calc_sc(net, ip=True)

In [ ]:
display(net.sgen)

In [ ]:
display(net.res_bus_sc)